In [53]:
from __future__ import print_function    # (at top of module)
import warnings
warnings.filterwarnings('ignore')
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]
from matplotlib.pyplot import figure
import math
import seaborn as sns
import io, os, sys, types

### This notebook is used to save the models to a file as a pickle representation so they can be loaded later into the web app.

In [54]:
# Read the data from the file
data = pd.read_csv('Data/data_500_entries_youtube.csv')
print("Number of entries in original data: " + str(len(data.index)))
data.head()

Number of entries in original data: 570


,song_id,song_title,artist,popularity,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration,loudness,valence,mode,youtube_view_count,youtube_video_title
0,spotify:track:5ygDXis42ncn6kYG14lEVG,Baby Shark,[Pinkfong],77,0.840,0.3410,115.062,0.2270,0.2450,0.000000,4,0.825,7,96333,-3.651,0.520,1,1956582159,Baby Shark Dance | Sing and Dance! | Animal So...
1,spotify:track:7fa9MBXhVfQ8P8Df9OEbD8,Girls Like You (feat. Cardi B),"[Maroon 5, Cardi B]",86,0.541,0.1300,124.959,0.0505,0.5680,0.000000,4,0.851,0,235545,-6.825,0.448,1,1300452389,Maroon 5 - Girls Like You ft. Cardi B
2,spotify:track:6De0lHrwBfPfrhorm9q1Xl,Me Rehúso,[Danny Ocean],83,0.804,0.0494,104.823,0.0677,0.0231,0.000000,4,0.744,1,205715,-6.327,0.426,1,1229501096,Danny Ocean - Me Rehúso (Official Audio)
3,spotify:track:1j6xOGusnyXq3l6IryKF3G,Déjala Que Vuelva (feat. Manuel Turizo),"[Piso 21, Manuel Turizo]",74,0.788,0.0753,170.019,0.0785,0.0482,0.000000,4,0.681,1,220117,-4.323,0.839,1,1216075058,Piso 21 - Déjala Que Vuelva (feat. Manuel Turi...
4,spotify:track:2ijef6ni2amuunRoKTlgww,Sin Pijama,"[Becky G, Natti Natasha]",90,0.745,0.1040,94.014,0.0464,0.3540,0.000029,4,0.791,11,188560,-3.695,0.820,0,1071141995,Becky G Natti Natasha - Sin Pijama (Video Ofic...


In [55]:
from project_modules import *

#final_data = label_data_combined(data, 90, 1000000000)
final_data = label_data_yt(data, 89, 1000000000)

Number of popular examples after thresholding :  59
Number of not popular examples after thresholding :  511


In [56]:
# Drop unnecessary columns from original data - also some legacy support for a different data format
if 'song_id' in data.columns:
    final_data.drop(['song_id', 'song_title', 'artist', 'popularity', 'youtube_view_count', 'youtube_video_title'], 1, inplace=True)
else:
    final_data.drop(['song_title', 'artist', 'popularity'], 1, inplace=True) 

In [57]:
# X will be our examples and y will be our labels
X = final_data.drop('is_popular', axis=1)
y = final_data['is_popular']
# Sanity checks
print("Number of entries in actual data: " + str(len(X.index)))
print("Number of entries in label data: " + str(len(y.index)))
X.head()

Number of entries in actual data: 570
Number of entries in label data: 570


,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration,loudness,valence,mode
0,0.840,0.3410,115.062,0.2270,0.2450,0.000000,4,0.825,7,96333,-3.651,0.520,1
1,0.541,0.1300,124.959,0.0505,0.5680,0.000000,4,0.851,0,235545,-6.825,0.448,1
2,0.804,0.0494,104.823,0.0677,0.0231,0.000000,4,0.744,1,205715,-6.327,0.426,1
3,0.788,0.0753,170.019,0.0785,0.0482,0.000000,4,0.681,1,220117,-4.323,0.839,1
4,0.745,0.1040,94.014,0.0464,0.3540,0.000029,4,0.791,11,188560,-3.695,0.820,0


In [58]:
from sklearn import preprocessing
import pickle
COLUMNS_TO_SCALE = ["energy", "liveness", "tempo", 
                    "speechiness", "acousticness", "instrumentalness", 
                    "time_signature", "danceability", "key", 
                    "duration", "loudness", "valence", "mode"]

In [59]:
from sklearn.neural_network import MLPClassifier
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from collections import Counter

In [60]:
print(Counter(y))
sm = SMOTE()
X_res, y_res = sm.fit_resample(X, y)
print(Counter(y_res))

Counter({0: 511, 1: 59})
Counter({1: 511, 0: 511})


In [25]:
# Model 1
# For each with less features we need own scaler which we need to export
#['energy', 'tempo', 'instrumentalness', 'danceability', 'loudness', 'valence'
scaler1 = preprocessing.StandardScaler()
scaler1.fit(X[['energy', 'tempo', 'instrumentalness', 'danceability', 'loudness', 'valence']])
pickle.dump(scaler1, open("3yp_scaler_log.pkl","wb"))
# Copy data back
X_log = scaler1.transform(X[['energy', 'tempo', 'instrumentalness', 'danceability', 'loudness', 'valence']])

model1 = LogisticRegression(solver='lbfgs',class_weight='balanced', random_state=3)
model1.fit(X_log, y)

pickle.dump(model1, open("3yp_log.pkl", "wb"))

C:\Users\Rares\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: ResourceWarning: unclosed file <_io.BufferedWriter name='3yp_scaler_log.pkl'>
  
C:\Users\Rares\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='3yp_log.pkl'>
  del sys.path[0]


In [11]:
# Model 2
#['energy', 'tempo', 'speechiness', 'instrumentalness', 'time_signature', 'duration', 'loudness']
scaler2 = preprocessing.StandardScaler()
scaler2.fit(X[['energy', 'tempo', 'speechiness', 'instrumentalness', 'time_signature', 'duration', 'loudness']])
pickle.dump(scaler2, open("3yp_scaler_svm.pkl","wb"))
# Copy data back
X_svm = scaler2.transform(X[['energy', 'tempo', 'speechiness', 'instrumentalness', 'time_signature', 'duration', 'loudness']])

model2 = svm.SVC(probability=True, gamma='scale', class_weight='balanced')
model2.fit(X_svm, y)

pickle.dump(model2, open("3yp_svm.pkl", "wb"))

C:\Users\Rares\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Rares\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: ResourceWarning: unclosed file <_io.BufferedWriter name='3yp_scaler_svm.pkl'>
  """
C:\Users\Rares\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys
C:\Users\Rares\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: ResourceWarning: unclosed file <_io.BufferedWriter name='3yp_svm.pkl'>
  if sys.path[0] == '':


In [38]:
# Model 3
# all

scaler3 = preprocessing.StandardScaler()
scaler3.fit(X)
pickle.dump(scaler3, open("3yp_scaler_mlp_smote.pkl","wb"))
# Copy data back
X_mlp_smote = scaler3.transform(X)
model3 = make_pipeline_imb(SMOTE(), MLPClassifier(solver="lbfgs", activation="relu", alpha=1, learning_rate="constant")
model3.fit(X_mlp_smote,y)

pickle.dump(model3, open("3yp_mlp_smote.pkl", "wb"))

C:\Users\Rares\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Rares\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: ResourceWarning: unclosed file <_io.BufferedWriter name='3yp_scaler_mlp_smote.pkl'>
  
C:\Users\Rares\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\Rares\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='3yp_mlp_smote.pkl'>
  del sys.path[0]


In [14]:
# Model 4
# all
# Try to also fit with oversampled data

scaler4 = preprocessing.StandardScaler()
scaler4.fit(X)
pickle.dump(scaler4, open("3yp_scaler_mlp_online.pkl","wb"))
X_mlp_online = scaler4.transform(X)

print(Counter(y))
sm2 = SMOTE()
X_res_mlp, y_res_mlp = sm2.fit_resample(X_mlp_online, y)
print(Counter(y_res))

model4 = MLPClassifier(activation = 'relu', solver='sgd', alpha=0.0001, learning_rate="constant")
model4.fit(X_res_mlp, y_res_mlp)

pickle.dump(model4, open("3yp_mlp_online.pkl", "wb"))

C:\Users\Rares\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Rares\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: ResourceWarning: unclosed file <_io.BufferedWriter name='3yp_scaler_mlp_online.pkl'>
  import sys
C:\Users\Rares\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


Counter({0: 511, 1: 59})
Counter({1: 511, 0: 511})


C:\Users\Rares\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Rares\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: ResourceWarning: unclosed file <_io.BufferedWriter name='3yp_mlp_online.pkl'>


In [ ]:
# Model 5
# ['energy', 'liveness', 'tempo', 'acousticness', 'instrumentalness', 'time_signature', 'duration', 'loudness', 'valence']
scaler5 = preprocessing.StandardScaler()
scaler5.fit(X)
pickle.dump(scaler5, open("3yp_scaler_mlp_simple.pkl","wb"))
# Copy data back
X_mlp_simple = scaler5.transform(X)
model5 = MLPClassifier(learning_rate="constant", solver="adam", alpha=0.001)
model5.fit(X_mlp_simple,y)

pickle.dump(model5, open("3yp_mlp_simple.pkl", "wb"))

In [62]:
# Model 6
# make_pipeline_imb(SMOTE(random_state=4), svm.SVC(probability=True, gamma='scale', random_state=3))
# feature_set :['energy', 'tempo', 'speechiness', 'loudness', 'valence']
scaler6 = preprocessing.StandardScaler()
scaler6.fit(X[['energy', 'tempo', 'speechiness', 'loudness', 'valence']])
pickle.dump(scaler6, open("3yp_scaler_svm_smote.pkl","wb"))
# Copy data back
X_svm_smote = scaler6.transform(X[['energy', 'tempo', 'speechiness', 'loudness', 'valence']])
model6 = make_pipeline_imb(SMOTE(random_state=4), svm.SVC(probability=True, gamma='scale'))
model6.fit(X_svm_smote,y)

pickle.dump(model6, open("3yp_svm_smote.pkl", "wb"))

In [27]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config

In [28]:
client_credentials_manager = SpotifyClientCredentials(config.client_id, config.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [29]:
test_song = sp.audio_features(["spotify:track:4kV4N9D1iKVxx1KLvtTpjS" 
                               "spotify:track:6ocbgoVGwYJhOv1GgI9NsF",
                               "spotify:track:5WHTFyqSii0lmT9R21abT8",
                               "spotify:track:2TIlqbIneP0ZY1O0EzYLlc",
                               "spotify:track:5itOtNx0WxtJmi1TQ3RuRd",
                               "spotify:track:1dAw715CaUd1HKGKXCzimK",
                               "spotify:track:5n2KsLTepK1vPeIkMw7UpV",
                               "spotify:track:6scFQGR5c6XYV33pcLbBIt",
                               "spotify:track:38df12R7YuZj8fIkhS3nRp",
                                  ])

In [31]:
test_song1 = []
for i in range(0, 8, 1):
    test_song1.append([test_song[i]['energy'], test_song[i]["liveness"], test_song[i]["tempo"], 
                    test_song[i]["speechiness"], test_song[i]["acousticness"], test_song[i]["instrumentalness"], 
                    test_song[i]["time_signature"], test_song[i]["danceability"], test_song[i]["key"], 
                    test_song[i]["duration_ms"], test_song[i]["loudness"], test_song[i]["valence"], test_song[i]["mode"]])
#     test_song1.append([test_song[i]['energy'], test_song[i]["tempo"], 
#                     test_song[i]["speechiness"],test_song[i]["instrumentalness"], 
#                     test_song[i]["time_signature"], 
#                     test_song[i]["duration_ms"], test_song[i]["loudness"]])
 #   X=X[['energy', 'tempo', 'speechiness', 'instrumentalness', 'time_signature', 'duration', 'loudness']]
test_df = pd.DataFrame(test_song1)

In [32]:
test_df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.317,0.0881,140.048,0.3340,0.592,0.000000,4,0.778,1,178627,-10.732,0.327,0
1,0.881,0.0793,98.994,0.1470,0.296,0.000003,4,0.674,9,178480,-2.853,0.234,1
2,0.405,0.1050,109.891,0.0319,0.751,0.000000,4,0.501,1,182161,-5.679,0.446,1
3,0.887,0.0811,122.015,0.0361,0.016,0.000503,4,0.807,1,229184,-4.311,0.606,0
4,0.501,0.1130,128.021,0.0309,0.818,0.000004,4,0.694,7,247812,-10.743,0.387,0
5,0.339,0.2980,76.908,0.0309,0.831,0.000000,4,0.595,10,168477,-8.527,0.434,1
6,0.821,0.1070,104.020,0.0371,0.582,0.000124,4,0.739,8,228162,-9.091,0.764,0
7,0.685,0.1920,138.980,0.0408,0.574,0.000037,4,0.771,4,252556,-8.439,0.310,0


In [34]:
print(test_song1)
data_np = scaler3.transform(test_df)
print(data_np)

[[0.317, 0.0881, 140.048, 0.334, 0.592, 0, 4, 0.778, 1, 178627, -10.732, 0.327, 0], [0.881, 0.0793, 98.994, 0.147, 0.296, 3.01e-06, 4, 0.674, 9, 178480, -2.853, 0.234, 1], [0.405, 0.105, 109.891, 0.0319, 0.751, 0, 4, 0.501, 1, 182161, -5.679, 0.446, 1], [0.887, 0.0811, 122.015, 0.0361, 0.016, 0.000503, 4, 0.807, 1, 229184, -4.311, 0.606, 0], [0.501, 0.113, 128.021, 0.0309, 0.818, 3.53e-06, 4, 0.694, 7, 247812, -10.743, 0.387, 0], [0.339, 0.298, 76.908, 0.0309, 0.831, 0, 4, 0.595, 10, 168477, -8.527, 0.434, 1], [0.821, 0.107, 104.02, 0.0371, 0.582, 0.000124, 4, 0.739, 8, 228162, -9.091, 0.764, 0], [0.685, 0.192, 138.98, 0.0408, 0.574, 3.73e-05, 4, 0.771, 4, 252556, -8.439, 0.31, 0]]
[[-2.11853672 -0.57815836  0.58641644  1.89685506  1.48318145 -0.14464198
   0.06047355  0.57788207 -1.17912291 -0.61245712 -2.1255199  -0.90535774
  -1.16003406]
 [ 1.38941558 -0.63492376 -0.80943875  0.15248772  0.19389792 -0.14460045
   0.06047355 -0.2139092   1.05469531 -0.61620692  1.46311546 -1.3382673

C:\Users\Rares\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [41]:
model7 = pickle.load(open("Webserver/models/3yp_mlp_smote.pkl","rb"))
predicted_labels = model7.predict(data_np)
print(predicted_labels)
print()
print()
predicted_probabilities = model7.predict_proba(data_np)
print(predicted_probabilities)
print()
print()
for i in range(0, 8, 1):
    #print("Predicted label: ", model.predict(data_np))
    print(predicted_labels[i])
    print(predicted_probabilities[i])
    #print("Class 0 probability: ",model.predict_proba(data_np[i])[0][0]," Class 1 probability: ",model.predict_proba(data_np[i])[0][1])

[1 1 1 1 0 1 0 0]


[[2.73006746e-01 7.26993254e-01]
 [1.78814492e-01 8.21185508e-01]
 [5.10552412e-02 9.48944759e-01]
 [2.73006746e-01 7.26993254e-01]
 [9.99120420e-01 8.79580266e-04]
 [2.73006746e-01 7.26993254e-01]
 [1.00000000e+00 1.02141802e-12]
 [9.97938045e-01 2.06195505e-03]]


1
[0.27300675 0.72699325]
1
[0.17881449 0.82118551]
1
[0.05105524 0.94894476]
1
[0.27300675 0.72699325]
0
[9.99120420e-01 8.79580266e-04]
1
[0.27300675 0.72699325]
0
[1.00000000e+00 1.02141802e-12]
0
[0.99793804 0.00206196]


C:\Users\Rares\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ResourceWarning: unclosed file <_io.BufferedReader name='Webserver/models/3yp_mlp_smote.pkl'>
  """Entry point for launching an IPython kernel.


In [50]:
model7.steps

[('smote',
  SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
     out_step='deprecated', random_state=None, ratio=None,
     sampling_strategy='auto', svm_estimator='deprecated')),
 ('mlpclassifier',
  MLPClassifier(activation='relu', alpha=1, batch_size='auto', beta_1=0.9,
         beta_2=0.999, early_stopping=False, epsilon=1e-08,
         hidden_layer_sizes=(5, 2), learning_rate='constant',
         learning_rate_init=0.001, max_iter=200, momentum=0.9,
         n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
         random_state=None, shuffle=True, solver='lbfgs', tol=0.0001,
         validation_fraction=0.1, verbose=False, warm_start=False))]